<img align="left" src="https://www.taborelec.com/Pictures/logo.png">

In [14]:
import os
import sys
srcpath = os.path.realpath('../SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst

In [15]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

In [16]:
pid = os.getpid()
print('process id {0}'.format(pid))

# Open a single-slot instrument:    
# Use lib_dir_path = None 
# for default location (C:\Windows\System32)

lib_dir_path = None

admin = TepAdmin(lib_dir_path)

# Get list of available PXI slots
slot_ids = admin.get_slot_ids()

# Assume that at least one slot was found
sid = slot_ids[0]

inst = admin.open_instrument(slot_id=sid)

# Change the default paranoia-level (0, 1, 2)
# from normal (1) to high (2). This is good for debugging
# because SYSTEM:ERROR is checked after each SCPI command.
inst.default_paranoia_level = 2

# Send query
resp = inst.send_scpi_query('*IDN?')
print('Connected to: ' + resp)

# Send command
inst.send_scpi_cmd(':INST:CHAN 1; :OUTP ON')



process id 18456
Connected to: Tabor Electronics,P2584M,000000000000,0.0.0


0

In [17]:

# Get the model name
resp = inst.send_scpi_query(":SYSTem:INFormation:MODel?")
print("Model: " + resp)

# Infer the natural DAC waveform format
if 'P9082' in resp:
    dac_mode = 8
else:
    dac_mode = 16
print("DAC waveform format: {0} bits-per-point".format(dac_mode))

# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))

max_dac = 2 ** dac_mode - 1
half_dac = max_dac / 2.0
    

Model: P2584M
DAC waveform format: 16 bits-per-point
Number of channels: 4
Max segment number: 65536
Available memory per DDR: 1,065,353,216 wave-bytes


In [18]:
seglen=102400000
num_cycles=2000
cyclelen = seglen / num_cycles

x = np.linspace(start=0, stop=2 * np.pi * num_cycles, num=seglen, endpoint=False)
y = (np.sin(x) + 1.0) * half_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
if dac_mode == 16:
    wave = y.astype(np.uint16)
else:
    wave = y.astype(np.uint8)
        
# data-type code: 'B' = uint8, 'H' = uint16
data_type = 'H' if dac_mode == 16 else 'B'


channb=1
segnum=1

cmd = ':INST:CHAN {0}'.format(channb)
inst.send_scpi_cmd(cmd)

cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
inst.send_scpi_cmd(cmd)

cmd = ':TRAC:SEL {0}'.format(segnum)
inst.send_scpi_cmd(cmd)

inst.timeout = 30000

#start_time = time.time()
tic = time.perf_counter()

inst.write_binary_data(':TRAC:DATA', wave)

#end_time = time.time()
#time_lapsed = end_time - start_time

#print('Downloaded the wave in "{0}" seconds'.format(time_lapsed))

toc = time.perf_counter()
timer = float(toc-tic)

print('Downloaded the wave in "{0}" seconds'.format(timer))


resp = inst.send_scpi_query(':SYST:ERR?')
resp = resp.rstrip()
if not resp.startswith('0'):
     print('ERROR: "{0}" after writing binary values'.format(resp))

cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum)
inst.send_scpi_cmd(cmd)

Downloaded the wave in "8.879999995770049e-05" seconds


0

In [13]:
# close connection
inst.close_instrument()
admin.close_inst_admin()

0